In [1]:
import cv2
import numpy  as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def display(window_name, canvas):
    cv2.imshow(window_name, canvas)
    cv2.waitKey(0)
    cv2.destroyWindow(window_name)

In [3]:
#defining colors
white = (255,255,255)
black = (0,0,0)
green = (0,255,0)
blue = (255, 0,0)
red = (0,0,255)

# Convolutions with OpenCV

<details>
  <summary> ⭐⭐⭐⭐ OpenCV Image Convolutions (CLICK) </summary>


<p>
In reality, an (image) convolution is simply an element-wise multiplication of two matrices followed by a sum. <br>
    1 . Take two matrices (which both have the same dimensions). <br>
    2 . Multiply them, element-by-element (i.e., not the dot-product, just a simple multiplication). <br>
    3 . Sum the elements together. <br> <br>
    Think of an image as a big matrix and kernel or convolutional matrix as a tiny matrix that is used for blurring, sharpening, edge detection, and other image processing functions. This tiny kernel sits on top of the big image and slides from left-to-right and top-to-bottom, applying a mathematical operation (i.e., a convolution) at each (x, y)-coordinate of the original image. <br> <br>
    Kernels can be an arbitrary size of M x N pixels, provided that both M and N are odd integers. <br>
    1. Select an (x, y)-coordinate from the original image. <br>
    2. Place the center of the kernel at this (x, y)-coordinate. <br>
    3. Take the element-wise multiplication of the input image region and the kernel, then sum up the values of these multiplication operations into a single value. The sum of these multiplications is called the kernel output. <br>
    4. Use the same (x, y)-coordinates from Step #1, but this time, store the kernel output in the same (x, y)-location as the output image. <br>
</p>
  
</details>



In [4]:
import skimage
from skimage.exposure import rescale_intensity

In [5]:
def convolve(image, kernel):
    # grab the spatial dimensions of the image, along with
    # the spatial dimensions of the kernel
    (iH, iW) = image.shape[:2]
    (kH, kW) = kernel.shape[:2]
    
    # allocate memory for the output image, taking care to
    # "pad" the borders of the input image so the spatial
    # size (i.e., width and height) are not reduced
    pad = (kW - 1) // 2
    
    image = cv2.copyMakeBorder(image, pad, pad, pad, pad, cv2.BORDER_REPLICATE)
    # cv2.BORDER_REPLICATE: It replicates the last element. Suppose, if image contains 
    # letters “abcdefgh” then output will be “aaaaa|abcdefgh|hhhhh“. 
    
    output = np.zeros((iH, iW), dtype="float32")
    
    # loop over the input image, "sliding" the kernel across
    # each (x, y)-coordinate from left-to-right and top to
    # bottom
    for y in np.arange(pad, iH + pad):
        for x in np.arange(pad, iW + pad):
            # extract the ROI of the image by extracting the
            # *center* region of the current (x, y)-coordinates dimensions
            roi = image[y - pad:y + pad + 1, x - pad:x + pad + 1]
            # perform the actual convolution by taking the
            # element-wise multiplicate between the ROI and
            # the kernel, then summing the matrix
            k = (roi * kernel).sum()
            # store the convolved value in the output (x,y)-
            # coordinate of the output image
            output[y - pad, x - pad] = k
    
    # rescale the output image to be in the range [0, 255]
    output = rescale_intensity(output, in_range=(0, 255))
    output = (output * 255).astype("uint8")
    # return the output image
    return output



In [6]:
smallBlur = np.ones((7, 7), dtype="float") * (1.0 / (7 * 7))
largeBlur = np.ones((21, 21), dtype="float") * (1.0 / (21 * 21))

In [12]:
# construct a sharpening filter
sharpen = np.array(([0, -1, 0], [-1, 5, -1], [0, -1, 0]), dtype="int")

In [13]:
# construct the Laplacian kernel used to detect edge-like regions of an image
laplacian = np.array(( [0, 1, 0], [1, -4, 1], [0, 1, 0]), dtype="int")

In [14]:
# construct the Sobel x-axis kernel
sobelX = np.array(( [-1, 0, 1], [-2, 0, 2], [-1, 0, 1]), dtype="int")

In [15]:
# construct the Sobel y-axis kernel
sobelY = np.array(( [-1, -2, -1], [0, 0, 0], [1, 2, 1]), dtype="int")

In [16]:
# construct the kernel bank, a list of kernels we're going
# to apply using both our custom `convole` function and
# OpenCV's `filter2D` function
kernelBank = (
    ("small_blur", smallBlur),
    ("large_blur", largeBlur),
    ("sharpen", sharpen),
    ("laplacian", laplacian),
    ("sobel_x", sobelX),
    ("sobel_y", sobelY)
)

In [17]:
# load the input image and convert it to grayscale
image = cv2.imread("batman.jpg", 1)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# loop over the kernels
for (kernelName, kernel) in kernelBank:
    # apply the kernel to the grayscale image using both our custom `convole` function and OpenCV's `filter2D` function
    
    print("[INFO] applying {} kernel".format(kernelName))
    convoleOutput = convolve(gray, kernel)
    opencvOutput = cv2.filter2D(gray, -1, kernel)
    
    # show the output images
    cv2.imshow("original", gray)
    cv2.imshow("{} - convole".format(kernelName), convoleOutput)
    cv2.imshow("{} - opencv".format(kernelName), opencvOutput)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

[INFO] applying small_blur kernel
[INFO] applying large_blur kernel
[INFO] applying sharpen kernel
[INFO] applying laplacian kernel
[INFO] applying sobel_x kernel
[INFO] applying sobel_y kernel


# OpenCV Morphological Operations

<details>
  <summary> ⭐⭐⭐⭐ OpenCV Morphological Operations (CLICK) </summary>


<p>
    Morphological operations are simple transformations applied to binary or grayscale images. 
    We can use morphological operations to increase the size of objects in images as well as decrease them. We can 
    also utilize morphological operations to close gaps between objects as well as open them. Like in image kernels, the structuring element slides from left-to-right and top-to-bottom for each pixel in the image. 
    
    Morphological operations “probe” an image with a structuring element. Like in image kernels, the structuring element slides from left-to-right and top-to-bottom for each pixel in the image. 
</p>
  
</details>

In [22]:
import cv2
import numpy as np

In [5]:
logo = cv2.imread("my_logo.jpg", 1)
gray = cv2.cvtColor(logo, cv2.COLOR_BGR2GRAY)
display("gray", gray)

In [24]:
logo.shape

(200, 1000)

### Erosion
Erosion works by defining a structuring element and then sliding this structuring element from left-to-right and top-to-bottom across the input image. A foreground pixel in the input image will be kept only if all pixels inside the structuring element are > 0. Otherwise, the pixels are set to 0 (i.e., background).

The second argument to cv2.erode is the structuring element. If this value is None, then a 3×3 structuring element, identical to the 8-neighborhood structuring element we saw above will be used. Of course, you could supply your own custom structuring element here instead of None as well.

In [31]:
eroded = cv2.erode(gray.copy(), None, )
display("eroded", eroded)

In [13]:
for i in range(0, 3):
    eroded = cv2.erode(gray.copy(), None, iterations=i + 1)
    display("eroded", eroded)
    
# The for loop controls the number of times, or iterations, we are going to apply the erosion. As the number of 
# erosions increases, the foreground logo will start to “erode” and disappear.

### Dilation
Dilations increase the size of foreground objects and are especially useful for joining broken parts of an image together. Dilations, just as an erosion, also utilize structuring elements — a center pixel p of the structuring element is set to white if ANY pixel in the structuring element is > 0.

Unlike an erosion where the foreground region is slowly eaten away at, a dilation actually grows our foreground region.

In [30]:
dilated = cv2.dilate(gray.copy(), None,)
display("dilated", dilated)

In [14]:
for i in range(0,3):
    dilated = cv2.dilate(gray.copy(), None, iterations=i+1)
    display("dilated", dilated)

### Opening

An opening is an erosion followed by a dilation.

Performing an opening operation allows us to remove small blobs from an image: first an erosion is applied to remove the small blobs, then a dilation is applied to regrow the size of the original object.

In [36]:
logo1 = cv2.imread("my_logo1.jpg", 1)
logo1 = cv2.cvtColor(logo1, cv2.COLOR_BGR2GRAY)
display("logo1", logo1)

In [52]:
kernelsizes = [(5,5), (7,7), (13,13)]
# initialize a list of kernels sizes that will be applied to the image

In [67]:
for kernelsize in kernelsizes:
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, kernelsize)    
    # cv2.MORPH_RECT, cv2.MORPH_ELLIPSE, cv2.MORPH_CROSS
    # The cv2.getStructuringElement function requires two arguments: the first is the type of structuring element we 
    # want, and the second is the size of the structuring element
    
    opening = cv2.morphologyEx(logo1.copy(), cv2.MORPH_OPEN, kernel)
    # The first required argument of cv2.morphologyEx is the image we want to apply the morphological operation to. 
    # The second argument is the actual type of morphological operation — in this case, it’s an opening operation. 
    # The last required argument is the kernel/structuring element that we are using.
    
    display("opening {}, {}".format(kernelsize[0], kernelsize[1]), opening)

### Closing
The exact opposite to an opening would be a closing. A closing is a dilation followed by an erosion.

As the name suggests, a closing is used to close holes inside of objects or for connecting components together.

In [56]:
logo2 = cv2.imread("my_logo2.jpg",1)
logo2 = cv2.cvtColor(logo2, cv2.COLOR_BGR2GRAY)
display("logo2", logo2)

In [65]:
kernelsizes= [(3,3), (9,9), (13,13)]

In [66]:
for kernelsize in kernelsizes:
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernelsize)
    closing = cv2.morphologyEx(logo2.copy(), cv2.MORPH_CLOSE, kernel)
    display("closing {}, {}".format(kernelsize[0], kernelsize[1]), closing)

### Morphological Gradient

A morphological gradient is the difference between a dilation and erosion. It is useful for determining the outline of a particular object of an image. 

In [68]:
logo3  = cv2.imread("my_logo2.jpg",1)
logo3 = cv2.cvtColor(logo3, cv2.COLOR_BGR2GRAY)
display("logo3", logo3)

In [69]:
kernelsizes = [(3,3), (5,5), (7,7)]

In [70]:
for kernelsize in kernelsizes:
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernelsize)
    
    gradient = cv2.morphologyEx(logo3.copy(), cv2.MORPH_GRADIENT, kernel)
    display("gradient {}, {}".format(kernelsize[0], kernelsize[1],), gradient)

### Top Hat/White Hat
A top hat (also known as a white hat) morphological operation is the difference between the original (grayscale/single channel) input image and the opening.

A top hat operation is used to reveal bright regions of an image on dark backgrounds.

Up until this point we have only applied morphological operations to binary images. But we can also apply morphological operations to grayscale images as well. In fact, both the top hat/white hat and the black hat operators are more suited for grayscale images rather than binary ones.

In [71]:
car = cv2.imread("car.jpg", 1)
gray = cv2.cvtColor(car, cv2.COLOR_BGR2GRAY)
display("car", gray)

In [75]:
kernelSize = (13,5)
# we define a rectangular structuring element with a width of 13 pixels and a height of 5 pixels. Structuring 
# elements can be of arbitrary size. And in this case, we are applying a rectangular element that is almost 3x wider 
# than it is tall. Because a license plate is roughly 3x wider than it is tall!

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernelSize)

tophat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, kernel)
display("tophat", tophat)

blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)
display("blackhat", blackhat)

## OpenCV Smoothing and Blurring

<details>
  <summary> ⭐⭐⭐⭐ OpenCV Smoothing & Blurring (CLICK) </summary>


<p>
    Practically, this means that each pixel in the image is mixed in with its surrounding pixel intensities. This “mixture” of pixels in a neighborhood becomes our blurred pixel.
    
By smoothing an image prior to applying techniques such as edge detection or thresholding we are able to reduce the amount of high-frequency content, such as noise and edges (i.e., the “detail” of an image).
</p>
  
</details>

### Average blurring ( cv2.blur )

In [28]:
# USAGE
# python blurring.py

# import the necessary packages
import cv2



# load the image, display it to our screen, and initialize a list of
# kernel sizes (so we can evaluate the relationship between kernel
# size and amount of blurring)
image = cv2.imread("batman.jpg",1)
cv2.imshow("Original", image)
kernelSizes = [(3, 3), (9, 9), (15, 15)]

# loop over the kernel sizes
for (kX, kY) in kernelSizes:
	# apply an "average" blur to the image using the current kernel
	# size
	blurred = cv2.blur(image, (kX, kY))
	cv2.imshow("Average ({}, {})".format(kX, kY), blurred)
	cv2.waitKey(0)

# close all windows to cleanup the screen
cv2.destroyAllWindows()
cv2.imshow("Original", image)

# loop over the kernel sizes again
for (kX, kY) in kernelSizes:
	# apply a "Gaussian" blur to the image
	blurred = cv2.GaussianBlur(image, (kX, kY), 0)
	cv2.imshow("Gaussian ({}, {})".format(kX, kY), blurred)
	cv2.waitKey(0)

# close all windows to cleanup the screen
cv2.destroyAllWindows()
cv2.imshow("Original", image)

# loop over the kernel sizes a final time
for k in (3, 9, 15):
	# apply a "median" blur to the image
	blurred = cv2.medianBlur(image, k)
	cv2.imshow("Median {}".format(k), blurred)
	cv2.waitKey(0)
	cv2.destroyAllWindows()

In [ ]:
# USAGE
# python bilateral.py

# import the necessary packages
import argparse
import cv2

# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", type=str, default="adrian.png",
	help="path to input image")
args = vars(ap.parse_args())

# load the image, display it to our screen, and construct a list of
# bilateral filtering parameters that we are going to explore
image = cv2.imread(args["image"])
cv2.imshow("Original", image)
params = [(11, 21, 7), (11, 41, 21), (11, 61, 39)]

# loop over the diameter, sigma color, and sigma space
for (diameter, sigmaColor, sigmaSpace) in params:
	# apply bilateral filtering to the image using the current set of
	# parameters
	blurred = cv2.bilateralFilter(image, diameter, sigmaColor, sigmaSpace)

	# show the output image and associated parameters
	title = "Blurred d={}, sc={}, ss={}".format(
		diameter, sigmaColor, sigmaSpace)
	cv2.imshow(title, blurred)
	cv2.waitKey(0)